In [ ]:
# imports
import numpy as np
import matplotlib
from matplotlib import ticker
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
from matplotlib.pyplot import subplots
from multiprocessing import Pool, shared_memory
import os
import fabio
import lmfit


from tools.comparison import detectormaker_fitting, voxelgridmaker_fitting, slabmaker_fitting, rebin_qmap_and_mask, scale_offset_mask_qmap, mask_forbidden_pixels, plot_linecuts
from tools.comparison import mirror_qmap_positive_qxy_only, rebin_and_combine_qmaps, create_wedge_mask, mirror_qmap_fix_detgap, trim_sim_data, shift_peak, linear_polar
%matplotlib widget

# Run through of single simulation (must do this first before full LMfit)

### Define variables of slab, voxelgrid, and detectorgrid generation

In [ ]:
#Slabmaker variables
input_filepath = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/PM6_forwardsim/xyz_files_Oct2024/pdb_files/aHEQ100.pdb' 
x_size = 200 #lamella
y_size = 50 #pipi
z_size = 100
a = 85.920
b = 62.461
c = 40.115
alpha = 90
beta = 90
gamma = 90

# voxelgridmaker variables
r_voxel_size = 0.3
q_voxel_size = 0.02
max_q = 2.5
energy = 12700
fill_bkg = True
smooth = 50

#detectormaker variables
num_pixels = 250
angle_init_vals = (90, 90, 90)
angle_init_axs = ('psi', 'phi', 'psi')
psis = np.linspace(0,90,91)
psi_weights_path = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/PM6_forwardsim/resampled_polefigs/resampled_ints_PM65CN_91.npy'
phis = np.linspace(0,179,180)
phi_weights_path = None
thetas = np.linspace(0,1,1)
theta_weights_path = None

### Run simulation (usually ~2min)

In [ ]:
#sample calculation
coords_slab, elements_slab = slabmaker_fitting(input_filepath, x_size, y_size, z_size, a, b, c, alpha, beta, gamma)
iq, qx, qy, qz = voxelgridmaker_fitting(coords_slab, elements_slab, r_voxel_size, q_voxel_size, max_q, energy, num_cpus=None, fill_bkg=fill_bkg, smooth=smooth)
det_sum, det_h, det_v = detectormaker_fitting(iq, qx, qy, qz, num_pixels, max_q, angle_init_vals, angle_init_axs, psis, psi_weights_path, phis, phi_weights_path, thetas, theta_weights_path, mirror=True)

### Load experimental data and fitting mask (fitting mask can be made at bottom of notebook)

In [ ]:
#load experimental data, fitting mask
img_name = 'TC47'
# img_name = 'TC3'
exp_path = f'/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/shared_PM6_crystal_study/experimental_data/{img_name}/'
mask_path = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/PM6_forwardsim/fitting_mask/PM6mask_maxq2.npy'
img_path = f'{exp_path}{img_name}_th0.15_qmap.tif'
qxy_path = f'{exp_path}{img_name}_th0.15_qxy.txt'
qz_path = f'{exp_path}{img_name}_th0.15_qz.txt'
exp_img = fabio.open(img_path).data
exp_qxy = np.loadtxt(qxy_path)
exp_qz = np.loadtxt(qz_path)
mask = np.load(mask_path)

#fix detector gaps, only take positive qxy
exp_img, exp_qxy, exp_qz = mirror_qmap_positive_qxy_only(exp_img, exp_qxy, exp_qz)

#trim simulated data to bounds of experimental data
sim_int_trim, det_h_trim, det_v_trim = trim_sim_data(det_sum, det_h, det_v, exp_qxy, exp_qz)

#rebin experimental data to simulated data resolution
rebin_map, rebin_mask = rebin_qmap_and_mask(det_h_trim, det_v_trim, exp_img, exp_qxy, exp_qz, mask)
# mask rebinned experimental data
rebin_map[rebin_mask==1]=0


### Fit background and offset of simulated GIWAXS

In [ ]:
#define helper function to optimize scale and offset. very fast
def optimize_scale_offset(sim_map, rebin_map, rebin_mask):
    # Flatten arrays for least squares
    sim_flat = sim_map[rebin_mask == 0].ravel()
    rebin_flat = rebin_map[rebin_mask == 0].ravel()

    # Solve for scale and offset: minimize |scale * sim_flat + offset - rebin_flat|^2
    A = np.vstack([sim_flat, np.ones_like(sim_flat)]).T
    coeffs, _, _, _ = np.linalg.lstsq(A, rebin_flat, rcond=None)
    scale, offset = coeffs
    return scale, offset

pad_range = (1.05,1.55)
sim_int_trim_pad = shift_peak(sim_int_trim.copy(), det_h_trim, det_v_trim, 0.1, pad_range)

best_scale, best_offset = optimize_scale_offset(sim_int_trim_pad, rebin_map, rebin_mask)
print(best_scale)
print(best_offset)

In [ ]:

# Define the model function for scaling and offsetting sim_comp_map
def model(params, sim_int_trim, det_h_trim, det_v_trim, rebin_map, mask, pad_range):
    scale = params['scale']
    offset = params['offset']
    pad_width = params['pad_width']
    
    sim_int_trim_pad = shift_peak(sim_int_trim.copy(), det_h_trim, det_v_trim, pad_width, pad_range)
    
    # Apply scale and offset to sim_comp_map
    scaled_map = scale * sim_int_trim_pad + offset
    
    # Calculate residuals only where mask is 0
    residuals = (rebin_map - scaled_map) * (mask == 0)
    return residuals.ravel()  # Flatten to a 1D array for lmfit

# Initialize parameters
params = lmfit.Parameters()
params.add('scale', value=1, min=0)  # Initial guess for scale
params.add('offset', value=40, min=0)          # Initial guess for offset
params.add('pad_width', value=0.1, min=0.05, max=0.2, vary=False) # Initial guess for offset

pad_range = (1.05,1.55)
# Perform the fit using lmfit's minimize function
result = lmfit.minimize(model, params, args=(sim_int_trim, det_h_trim, det_v_trim, rebin_map, rebin_mask, pad_range), method='nelder')

# Display the fit report with best-fit parameters
lmfit.report_fit(result)

# Access the optimized scale and offset
best_scale = result.params['scale'].value
best_offset = result.params['offset'].value
best_width = result.params['pad_width'].value

### Plot simulated, experimental and residuals GIWAXS profiles

In [ ]:
pad_range = (1.05,1.55)
sim_int_trim_pad = shift_peak(sim_int_trim.copy(), det_h_trim, det_v_trim, 0.1, pad_range)

#scale and offset simulated data
sim_comp_map = scale_offset_mask_qmap(sim_int_trim_pad, rebin_mask, scale=best_scale, offset=best_offset)
#take difference between experimental and simulated data
diff_pct_map = np.where(rebin_map > 1e-10, (sim_comp_map - rebin_map), 0)

#plot results
fontsize = 12
cbar_shrink = 0.45
cbar_pad = 0.05
cbar_aspect=25
max_q = 2
fig,(ax2, ax1, ax3)=plt.subplots(1,3, figsize=(12,6))
cax = ax1.imshow(rebin_map,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(rebin_map, 1), vmax=np.percentile(rebin_map, 99.9)),
           extent=(np.min(det_h_trim),np.max(det_h_trim),np.min(det_v_trim),np.max(det_v_trim)),
           cmap='turbo',
           origin = 'lower')
cbar1 = fig.colorbar(cax, ax=ax1, shrink=cbar_shrink, aspect=cbar_aspect, pad=cbar_pad)
ax1.set_xlabel('$\mathregular{q_{xy}}$ ($\AA^{-1}$)',fontsize=fontsize)
ax1.set_ylabel('$\mathregular{q_z}$ ($\AA^{-1}$)',fontsize=fontsize)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax1.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax1.set_xlim(0,max_q)
ax1.set_ylim(0,max_q)
ax1.set_title('Experimental GIWAXS', fontsize=fontsize)

cax = ax2.imshow(sim_comp_map,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(rebin_map, 1), vmax=np.percentile(rebin_map, 99.9)),
           extent=(np.min(det_h_trim),np.max(det_h_trim),np.min(det_v_trim),np.max(det_v_trim)),
           cmap='turbo',
           origin = 'lower')
cbar2 = fig.colorbar(cax, ax=ax2, shrink=cbar_shrink, aspect=cbar_aspect, pad=cbar_pad)
ax2.set_xlabel('$\mathregular{q_{xy}}$ ($\AA^{-1}$)',fontsize=fontsize)
ax2.set_ylabel('$\mathregular{q_z}$ ($\AA^{-1}$)',fontsize=fontsize)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax2.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax2.set_xlim(0,max_q)
ax2.set_ylim(0,max_q)
ax2.set_title('Simulated GIWAXS', fontsize=fontsize)

cmap_bound = np.max([np.abs(np.percentile(diff_pct_map, 0.1)), np.abs(np.percentile(diff_pct_map, 99.9))])
cax = ax3.imshow(diff_pct_map,
           norm=matplotlib.colors.Normalize(vmin=-cmap_bound, vmax=cmap_bound),
        #    norm=matplotlib.colors.LogNorm(vmin=np.percentile(qmap_compare, 30), vmax=np.percentile(qmap_compare, 99.95)),
           extent=(np.min(det_h_trim),np.max(det_h_trim),np.min(det_v_trim),np.max(det_v_trim)),
           cmap='bwr',
           origin = 'lower')
cbar3 = fig.colorbar(cax, ax=ax3, shrink=cbar_shrink, aspect=cbar_aspect, pad=cbar_pad)
# cbar3.set_label('', fontsize=fontsize)  # Add label to the colorbar
ax3.set_xlabel('$\mathregular{q_{xy}}$ ($\AA^{-1}$)',fontsize=fontsize)
ax3.set_ylabel('$\mathregular{q_z}$ ($\AA^{-1}$)',fontsize=fontsize)
ax3.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax3.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax3.set_xlim(0,max_q)
ax3.set_ylim(0,max_q)
ax3.set_title('Sim. - Exp.', fontsize=fontsize)

plt.tight_layout()

In [ ]:
savepath = '/Users/Thomas2/Downloads/Figuretestgiwaxs.png'
suptitle = ''
plot_linecuts(savepath, rebin_map, sim_comp_map, det_h_trim, det_v_trim, suptitle)

# Full Lmfit to refine slab size (suggest to run this on dedicated computational resource)

In [ ]:
# Define the model function for scaling and offsetting sim_comp_map
def fullfit_model(params, fixed_slab_params, fixed_voxelgrid_params, fixed_detectormaker_params, fixed_exp_params):
    scale = params['scale']
    offset = params['offset']
    x_size = params['x_size']
    y_size = params['y_size']
    z_size = params['z_size']

    input_filepath, a, b, c, alpha, beta, gamma = fixed_slab_params
    r_voxel_size, q_voxel_size, max_q, energy, fill_bkg, smooth = fixed_voxelgrid_params
    num_pixels, angle_init_vals, angle_init_axs, psis, psi_weights_path, phis, phi_weights_path, thetas, theta_weights_path = fixed_detectormaker_params
    rebin_map, rebin_mask, exp_qxy, exp_qz, pad_width, pad_range = fixed_exp_params

    #generate simulated GIWAXS
    coords_slab, elements_slab = slabmaker_fitting(input_filepath, x_size, y_size, z_size, a, b, c, alpha, beta, gamma)
    iq, qx, qy, qz = voxelgridmaker_fitting(coords_slab, elements_slab, r_voxel_size, q_voxel_size, max_q, energy, num_cpus=None, fill_bkg=fill_bkg, smooth=smooth)
    det_sum, det_h, det_v = detectormaker_fitting(iq, qx, qy, qz, num_pixels, max_q, angle_init_vals, angle_init_axs, psis, psi_weights_path, phis, phi_weights_path, thetas, theta_weights_path, mirror=True)

    #trim simulated data to bounds of experimental data
    sim_int_trim, det_h_trim, det_v_trim = trim_sim_data(det_sum, det_h, det_v, exp_qxy, exp_qz)

    sim_int_trim_pad = shift_peak(sim_int_trim.copy(), det_h_trim, det_v_trim, pad_width, pad_range)
    
    # Apply scale and offset to sim_comp_map
    scaled_map = scale * sim_int_trim_pad + offset
    
    # Calculate residuals only where mask is 0
    residuals = (rebin_map - scaled_map) * (rebin_mask == 0)
    return residuals.ravel()  # Flatten to a 1D array for lmfit

# Initialize variable parameters
params = lmfit.Parameters()
params.add('scale', value=2, min=0)  # Initial guess for scale
params.add('offset', value=40, min=0, max=60)   # Initial guess for offset
params.add('x_size', value=150, min=100, max=300) 
params.add('y_size', value=150, min=100, max=300)  
params.add('z_size', value=200, min=100, max=300)

#set fixed params
#Slabmaker variables
input_filepath = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/PM6_forwardsim/xyz_files_Oct2024/pdb_files/aHEQ100.pdb' 
a = 85.920
b = 62.461
c = 40.115
alpha = 90
beta = 90
gamma = 90

# voxelgridmaker variables
r_voxel_size = 0.3
q_voxel_size = 0.04
max_q = 2.5
energy = 12700
fill_bkg = True
smooth = 50

#detectormaker variables
num_pixels = 250
angle_init_vals = (90, 90, 90)
angle_init_axs = ('psi', 'phi', 'psi')
psis = np.linspace(0,90,91)
psi_weights_path = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/PM6_forwardsim/resampled_polefigs/resampled_ints_PM65CN_91.npy'
phis = np.linspace(0,179,180)
phi_weights_path = None
thetas = np.linspace(0,1,1)
theta_weights_path = None

pad_width = 0.15008304
pad_range = (1.05,1.55)

fixed_slab_params = (input_filepath, a, b, c, alpha, beta, gamma)
fixed_voxelgrid_params = (r_voxel_size, q_voxel_size, max_q, energy, fill_bkg, smooth)
fixed_detectormaker_params = (num_pixels, angle_init_vals, angle_init_axs, psis, psi_weights_path, phis, phi_weights_path, thetas, theta_weights_path)
fixed_exp_params = (rebin_map, rebin_mask, exp_qxy, exp_qz, pad_width, pad_range)


# Perform the fit using lmfit's minimize function
result = lmfit.minimize(fullfit_model, params, args=(fixed_slab_params, fixed_voxelgrid_params, fixed_detectormaker_params, fixed_exp_params), max_nfev=25)

result.fit_report()
# Display the fit report with best-fit parameters
lmfit.report_fit(result)

# Access the optimized scale and offset
best_scale = result.params['scale'].value
best_offset = result.params['offset'].value
best_x_size = result.params['x_size'].value
best_y_size = result.params['y_size'].value
best_z_size = result.params['z_size'].value

best_params = (best_scale, best_offset, best_x_size, best_y_size, best_z_size)

### Evaulate fit, plot simulated, experimental and residuals

In [ ]:
def evaluate_fit(best_params,  fixed_slab_params, fixed_voxelgrid_params, fixed_detectormaker_params, fixed_exp_params):
   scale, offset, x_size, y_size, z_size = best_params
   input_filepath, a, b, c, alpha, beta, gamma = fixed_slab_params
   r_voxel_size, q_voxel_size, max_q, energy, fill_bkg, smooth = fixed_voxelgrid_params
   num_pixels, angle_init_vals, angle_init_axs, psis, psi_weights_path, phis, phi_weights_path, thetas, theta_weights_path = fixed_detectormaker_params
   rebin_map, rebin_mask, exp_qxy, exp_qz, pad_width, pad_range = fixed_exp_params

   #generate simulated GIWAXS
   coords_slab, elements_slab = slabmaker_fitting(input_filepath, x_size, y_size, z_size, a, b, c, alpha, beta, gamma)
   iq, qx, qy, qz = voxelgridmaker_fitting(coords_slab, elements_slab, r_voxel_size, q_voxel_size, max_q, energy, num_cpus=None, fill_bkg=fill_bkg, smooth=smooth)
   det_sum, det_h, det_v = detectormaker_fitting(iq, qx, qy, qz, num_pixels, max_q, angle_init_vals, angle_init_axs, psis, psi_weights_path, phis, phi_weights_path, thetas, theta_weights_path, mirror=True)

   #trim simulated data to bounds of experimental data
   sim_int_trim, det_h_trim, det_v_trim = trim_sim_data(det_sum, det_h, det_v, exp_qxy, exp_qz)

   sim_int_trim_pad = shift_peak(sim_int_trim.copy(), det_h_trim, det_v_trim, pad_width, pad_range)
       
   # Apply scale and offset to sim_comp_map
   scaled_map = scale * sim_int_trim_pad + offset

   sim_comp_map = scaled_map.copy()
   sim_comp_map[rebin_mask==1] = 0

   diff_pct_map = np.where(rebin_map > 1e-10, 100 * (sim_comp_map - rebin_map), 0)

   return rebin_map, sim_comp_map, diff_pct_map

rebin_map, sim_comp_map, diff_pct_map = evaluate_fit(best_params,  fixed_slab_params, fixed_voxelgrid_params, fixed_detectormaker_params, fixed_exp_params)

#plot results
fontsize = 12
cbar_shrink = 0.45
cbar_pad = 0.05
cbar_aspect=25
max_q = 2
fig,(ax2, ax1, ax3)=plt.subplots(1,3, figsize=(12,6))
cax = ax1.imshow(rebin_map,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(rebin_map, 1), vmax=np.percentile(rebin_map, 99.9)),
           extent=(np.min(det_h_trim),np.max(det_h_trim),np.min(det_v_trim),np.max(det_v_trim)),
           cmap='turbo',
           origin = 'lower')
cbar1 = fig.colorbar(cax, ax=ax1, shrink=cbar_shrink, aspect=cbar_aspect, pad=cbar_pad)
ax1.set_xlabel('$\mathregular{q_{xy}}$ ($\AA^{-1}$)',fontsize=fontsize)
ax1.set_ylabel('$\mathregular{q_z}$ ($\AA^{-1}$)',fontsize=fontsize)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax1.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax1.set_xlim(0,max_q)
ax1.set_ylim(0,max_q)
ax1.set_title('Experimental GIWAXS', fontsize=fontsize)

cax = ax2.imshow(sim_comp_map,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(rebin_map, 1), vmax=np.percentile(rebin_map, 99.9)),
           extent=(np.min(det_h_trim),np.max(det_h_trim),np.min(det_v_trim),np.max(det_v_trim)),
           cmap='turbo',
           origin = 'lower')
cbar2 = fig.colorbar(cax, ax=ax2, shrink=cbar_shrink, aspect=cbar_aspect, pad=cbar_pad)
ax2.set_xlabel('$\mathregular{q_{xy}}$ ($\AA^{-1}$)',fontsize=fontsize)
ax2.set_ylabel('$\mathregular{q_z}$ ($\AA^{-1}$)',fontsize=fontsize)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax2.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax2.set_xlim(0,max_q)
ax2.set_ylim(0,max_q)
ax2.set_title('Simulated GIWAXS', fontsize=fontsize)

cmap_bound = np.max([np.abs(np.min(diff_pct_map)), np.abs(np.max(diff_pct_map))])
cax = ax3.imshow(diff_pct_map,
           norm=matplotlib.colors.Normalize(vmin=-1*cmap_bound, vmax=cmap_bound),
        #    norm=matplotlib.colors.LogNorm(vmin=np.percentile(qmap_compare, 30), vmax=np.percentile(qmap_compare, 99.95)),
           extent=(np.min(det_h_trim),np.max(det_h_trim),np.min(det_v_trim),np.max(det_v_trim)),
           cmap='bwr',
           origin = 'lower')
cbar3 = fig.colorbar(cax, ax=ax3, shrink=cbar_shrink, aspect=cbar_aspect, pad=cbar_pad)
cbar3.set_label('(Percent)', fontsize=fontsize)  # Add label to the colorbar
ax3.set_xlabel('$\mathregular{q_{xy}}$ ($\AA^{-1}$)',fontsize=fontsize)
ax3.set_ylabel('$\mathregular{q_z}$ ($\AA^{-1}$)',fontsize=fontsize)
ax3.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax3.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax3.set_xlim(0,max_q)
ax3.set_ylim(0,max_q)
ax3.set_title('Sim. - Exp.', fontsize=fontsize)

plt.tight_layout()

# Side-by-side comparison of simulated data (make sure sim_det_ints is defined)

In [ ]:
#side by side comparison

img_name = 'TC47'
# img_name = 'TC3'
exp_path = f'/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/shared_PM6_crystal_study/experimental_data/{img_name}/'
img_path = f'{exp_path}{img_name}_th0.15_qmap.tif'
qxy_path = f'{exp_path}{img_name}_th0.15_qxy.txt'
qz_path = f'{exp_path}{img_name}_th0.15_qz.txt'
exp_img = fabio.open(img_path).data
exp_qxy = np.loadtxt(qxy_path)
exp_qz = np.loadtxt(qz_path)
# exp_img = np.flipud(exp_img)
# exp_qz = np.flip(exp_qz)
exp_img, exp_qxy, exp_qz = mirror_qmap_fix_detgap(exp_img, exp_qxy, exp_qz)

sim_det_ints = mask_forbidden_pixels(det_sum, det_h, det_v, 0.15, 12700) #Jonathan
# sim_det_ints = mask_forbidden_pixels(sim_det_ints, det_h, det_v, 0.1, 15000) #McNeill 
qmap_compare = rebin_and_combine_qmaps(exp_img, exp_qxy, exp_qz, sim_det_ints, det_h, det_v)
qmap_compare[qmap_compare<=0]=1
qmap_compare[qmap_compare!=qmap_compare]=1

fig,ax=plt.subplots(figsize=(10,5))
cax = ax.imshow(qmap_compare,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(qmap_compare, 1), vmax=np.percentile(qmap_compare, 99.5)),
        #    norm=matplotlib.colors.LogNorm(vmin=np.percentile(qmap_compare, 30), vmax=np.percentile(qmap_compare, 99.95)),
           extent=(np.min(exp_qxy),np.max(exp_qxy),np.min(exp_qz),np.max(exp_qz)),
           cmap='turbo',
           origin = 'lower')
cbar = fig.colorbar(cax, ax=ax, shrink=0.82, aspect=20, pad=0.02)
ax.set_xlabel('$\mathregular{q_{xy}}$ ($\AA^{-1}$)',fontsize=16)
ax.set_ylabel('$\mathregular{q_z}$ ($\AA^{-1}$)',fontsize=16)
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax.set_xlim(-2,2)
ax.set_ylim(0,2.0)
plt.subplots_adjust(left=0.1, right=0.8, top=0.9, bottom=0.1)

# Create Mask

In [ ]:
#load experimental data, create mask for fitting

img_name = 'TC47'
# img_name = 'TC3'
exp_path = f'/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/shared_PM6_crystal_study/experimental_data/{img_name}/'
img_path = f'{exp_path}{img_name}_th0.15_qmap.tif'
qxy_path = f'{exp_path}{img_name}_th0.15_qxy.txt'
qz_path = f'{exp_path}{img_name}_th0.15_qz.txt'
exp_img = fabio.open(img_path).data
exp_qxy = np.loadtxt(qxy_path)
exp_qz = np.loadtxt(qz_path)

exp_img, exp_qxy, exp_qz = mirror_qmap_positive_qxy_only(exp_img, exp_qxy, exp_qz)


#create qmap mask
mask = create_wedge_mask(exp_img, exp_qxy, exp_qz, 0.15, 12700)

# use meshgrids to define rest of mask
exp_qxy_mesh, exp_qz_mesh = np.meshgrid(exp_qxy, exp_qz)
exp_qr_mesh = np.sqrt(exp_qxy_mesh**2 + exp_qz_mesh**2)

#mask minimum q_value
min_qr = 0.15
mask[exp_qr_mesh<min_qr] = 1

#mask max q_value
max_qr = 2.0
mask[exp_qr_mesh>max_qr] = 1

#mask yoneda peak
min_qz = 0.033
mask[exp_qz_mesh<min_qz] = 1

#mask specular
width_pos_qxy = 0.06
height_qz = 0.25
mask[np.logical_and(exp_qz_mesh<height_qz, exp_qxy_mesh<width_pos_qxy)] = 1

masked_exp_img = np.copy(exp_img)
masked_exp_img[mask==1] = 0

fig,ax=plt.subplots(figsize=(10,5))
cax = ax.imshow(masked_exp_img,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(exp_img, 1), vmax=np.percentile(exp_img, 99.5)),
        #    norm=matplotlib.colors.LogNorm(vmin=np.percentile(qmap_compare, 30), vmax=np.percentile(qmap_compare, 99.95)),
           extent=(np.min(exp_qxy),np.max(exp_qxy),np.min(exp_qz),np.max(exp_qz)),
           cmap='turbo',
           origin = 'lower')
cbar = fig.colorbar(cax, ax=ax, shrink=1, aspect=20, pad=0.02)
ax.set_xlabel('$\mathregular{q_{xy}}$ ($\AA^{-1}$)',fontsize=16)
ax.set_ylabel('$\mathregular{q_z}$ ($\AA^{-1}$)',fontsize=16)
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax.set_xlim(0,2)
ax.set_ylim(0,2.0)
plt.subplots_adjust(left=0.1, right=0.8, top=0.9, bottom=0.1)

save_path = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/PM6_forwardsim/fitting_mask/PM6mask_maxq2.npy'
np.save(save_path, mask)